机器学习系统中主要的组成部分是数据处理流水线。     
在数据被输入到机器学习算法中进行训练之前，需要对数据做各种方式的处理，使得该数据可以被算法利用。
在构建一个准确的、可扩展的机器学习系统中，拥有一个健壮的数据处理流水线非常重要。
有很多基本的函数可以使用，通常数据处理流水线就是这些函数的组合。     
不推荐用嵌套或循环的方式调用这些函数，而是用函数式编程的方式构建函数组合。

In [10]:
# 函数组合
import numpy as np
from functools import reduce 

def add3(input_array):
    return map(lambda x: x+3, input_array)

def mul2(input_array):
    return map(lambda x: x*2, input_array)

def sub5(input_array):
    return map(lambda x: x-5, input_array)

def function_composer(*args):
    return reduce(lambda f, g: lambda x: f(g(x)), args)
#     函数将一个数据集合（链表，元组等）中的所有数据进行下列操作：
#     用传给 reduce 中的函数 function（有两个参数）先对集合中的第 1、2 个元素进行操作，
#     得到的结果再与第三个数据用 function 函数运算，最后得到一个结果


def test():
    arr = np.array([2,5,4,7])

    print("\nOperation: sub5(mul2(add3(arr)))")
    
    arr1 = add3(arr)
    arr2 = mul2(arr1)
    arr3 = sub5(arr2)
    print("Output using the lengthy way:", list(arr3))

    func_composed = function_composer(sub5, mul2, add3)
    print("Output using function composition:", list(func_composed(arr)))

    print("\nOperation: mul2(sub5(mul2(add3(sub5(arr)))))\nOutput:", \
            list(function_composer(mul2, sub5, mul2, add3, sub5)(arr)))
    
    
if __name__=='__main__':
    test()



Operation: sub5(mul2(add3(arr)))
Output using the lengthy way: [5, 11, 9, 15]
Output using function composition: [5, 11, 9, 15]

Operation: mul2(sub5(mul2(add3(sub5(arr)))))
Output: [-10, 2, -2, 10]


**机器学习流水线工作原理**   
- 选择k个最佳特征的方式是基于单变量的特征选择
- 选择过程是先进行变量统计测试，然后从特征向量中抽取最优秀的特征
- 做了这些测试后，向量空间的每个特征将有一个评价分数。基于这些评价分数，选择最好的k个特征
- 一旦抽取出k个特征，一个k维特征向量就形成了，可以将这个特征向量用于随机森林分类器的输入训练数据

In [16]:
# 机器学习流水线
# 包括预处理、特征选择、监督学习、非监督学习等函数
from sklearn.datasets import samples_generator
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline

# 生成一些示例数据
X, y = samples_generator.make_classification(
        n_informative=4, n_features=20, n_redundant=0, random_state=5)
# n_features :特征个数，n_informative：多信息特征的个数，n_redundant：冗余信息，informative特征的随机线性组合

# 特征选择器
selector_k_best = SelectKBest(f_regression, k=10)
# 随机森林分类器
classifier = RandomForestClassifier(n_estimators=50, max_depth=4)
# 构建机器学习流水线
pipeline_classifier = Pipeline([('selector', selector_k_best), ('rf', classifier)])
# 可以选择更新这些参数
pipeline_classifier.set_params(selector__k=6, 
        rf__n_estimators=25)
# 训练分类器
pipeline_classifier.fit(X, y)
# 预测输出结果
prediction = pipeline_classifier.predict(X)
print("\nPredictions:\n", prediction)
# 打印分类器得分
print("\nScore:", pipeline_classifier.score(X, y))

# 打印被分类器选中的特征
features_status = pipeline_classifier.named_steps['selector'].get_support()
selected_features = []
for count, item in enumerate(features_status):
    if item:
        selected_features.append(count)

print("\nSelected features (0-indexed):", ', '.join([str(x) for x in selected_features]))


Predictions:
 [1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 1
 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 1]

Score: 0.97

Selected features (0-indexed): 0, 5, 9, 10, 11, 15
